# Read **PARQUET** File

In [0]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)

In [0]:
df.show(truncate=False)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [0]:
df.write.parquet('/FileStore/tables/output/parquet/dummy.parquet')

In [0]:
dbutils.fs.ls('/FileStore/tables/output/parquet/')

Out[10]: [FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/', name='dummy.parquet/', size=0, modificationTime=0)]

In [0]:
df = spark.read.parquet('/FileStore/tables/output/parquet/dummy.parquet')

In [0]:
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|  Robert |          |Williams|42114|     M|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
| Michael |      Rose|        |40288|     M|  4000|
|   James |          |   Smith|36636|     M|  3000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



In [0]:
df.write.mode("append").parquet('/FileStore/tables/output/parquet/dummy.parquet')

In [0]:
df = spark.read.parquet('/FileStore/tables/output/parquet/dummy.parquet')

In [0]:
df_app = df.show(truncate=False)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|Robert   |          |Williams|42114|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Michael  |Rose      |        |40288|M     |4000  |
|James    |          |Smith   |36636|M     |3000  |
|James    |          |Smith   |36636|M     |3000  |
|James    |          |Smith   |36636|M     |3000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
|Jen      |Mary      |Brown   |     |F     |-1    |
|Jen      |Mary      |Brown   |     |F     |-1    |
|Jen      |Mary      |Brown   |     |F     |-1    |
|Maria    |A

In [0]:
spark.sql('CREATE TEMPORARY VIEW PERSON USING parquet OPTIONS (path \"/FileStore/tables/output/parquet/dummy.parquet")')
spark.sql('SELECT * FROM PERSON').show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3174752072356334>:1
----> 1 #df_app.createOrReplaceTempView("ParquetTable")
      2 parkSQL = spark.sql("select * from ParquetTable where salary >= 4000 ")
      3 parkSQL.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), 

In [0]:
%sql
SELECT DISTINCT * FROM PERSON

firstname,middlename,lastname,dob,gender,salary
Robert,,Williams,42114,M,4000
Maria,Anne,Jones,39192,F,4000
Michael,Rose,,40288,M,4000
James,,Smith,36636,M,3000
Jen,Mary,Brown,,F,-1


In [0]:
dbutils.fs.ls('/FileStore/tables/output/parquet/')

Out[50]: [FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/', name='dummy.parquet/', size=0, modificationTime=0)]

In [0]:
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|  Robert |          |Williams|42114|     M|  4000|
|  Robert |          |Williams|42114|     M|  4000|
|  Robert |          |Williams|42114|     M|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
| Michael |      Rose|        |40288|     M|  4000|
| Michael |      Rose|        |40288|     M|  4000|
| Michael |      Rose|        |40288|     M|  4000|
|   James |          |   Smith|36636|     M|  3000|
|   James |          |   Smith|36636|     M|  3000|
|   James |          |   Smith|36636|     M|  3000|
|      Jen|      Mary|   Brown|     |     F|    -1|
|      Jen|      Mary|   Brown|     |     F|    -1|
|      Jen|      Mary|   Brown|     |     F|    -1|
|      Jen|      Mary|   Brown|     |     F|    -1|
|   Maria | 

In [0]:
df.write.partitionBy('gender','salary').mode('overwrite').parquet('/FileStore/tables/output/parquet/dummy.parquet')

In [0]:
dbutils.fs.ls('/FileStore/tables/output/parquet/dummy.parquet')

Out[53]: [FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1756041027000),
 FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_committed_1087315736948335903', name='_committed_1087315736948335903', size=519, modificationTime=1756039312000),
 FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_committed_2487912758901360991', name='_committed_2487912758901360991', size=519, modificationTime=1756039355000),
 FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_committed_385036967555043728', name='_committed_385036967555043728', size=612, modificationTime=1756038933000),
 FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_committed_4257467560565963337', name='_committed_4257467560565963337', size=519, modificationTime=1756039328000),
 FileInfo(path='dbfs:/FileStore/tables/output/parquet/dummy.parquet/_committed_7198564735653840463', name='_committed_719856

In [0]:
df_par = spark.read.parquet('/FileStore/tables/output/parquet/dummy.parquet/gender=F')

In [0]:
df_par.show()

+---------+----------+--------+-----+------+
|firstname|middlename|lastname|  dob|salary|
+---------+----------+--------+-----+------+
|   Maria |      Anne|   Jones|39192|  4000|
|   Maria |      Anne|   Jones|39192|  4000|
|   Maria |      Anne|   Jones|39192|  4000|
|      Jen|      Mary|   Brown|     |    -1|
|      Jen|      Mary|   Brown|     |    -1|
|      Jen|      Mary|   Brown|     |    -1|
|      Jen|      Mary|   Brown|     |    -1|
|   Maria |      Anne|   Jones|39192|  4000|
+---------+----------+--------+-----+------+

